# Imports and Functions

In [15]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from lib.renormalization_np import track_rg_flow


def generate_rg_flow(J, M, n_max=20, n_steps=50, b=2, d=2):
    """
    Generate a single RG flow at specific J and M values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        
    Returns:
        Final coefficient matrix at fixed point (normalized)
    """
    # Track the RG flow
    flow_history = track_rg_flow(J, J, M, b, d, n_max, n_steps)
    
    # Get the final matrix
    final_matrix = flow_history[-1]
    
    # Normalize by (0,0) coefficient
    center_idx = n_max
    if abs(final_matrix[center_idx, center_idx]) > 1e-10:
        final_matrix = final_matrix / final_matrix[center_idx, center_idx]
    
    return final_matrix

def calculate_villain_coefficients(Jv, coeff_indices, n_max=20):
    """
    Calculate the Villain fixed-line coefficients for a given Jv
    
    Args:
        Jv: Villain coupling parameter
        coeff_indices: List of (n,m) tuples for desired coefficients
        n_max: Maximum Fourier mode
        
    Returns:
        Dictionary mapping (n,m) tuples to Villain coefficient values
    """
    villain_coeffs = {}
    
    for n, m in coeff_indices:
        # Calculate exponent based on coefficient indices
        exponent = n**2 + m**2
        
        # Calculate Villain coefficient
        villain_coeffs[(n, m)] = np.exp(-exponent/(2*abs(Jv)))
        
        # For odd indices and negative Jv, coefficients may be negative
        if (n + m) % 2 == 1 and Jv < 0:
            villain_coeffs[(n, m)] *= -1
    
    return villain_coeffs

def estimate_Jv_from_coefficients(final_matrix, n_max=20):
    """
    Estimate Jv from coefficient matrix using the ratio method
    
    Args:
        final_matrix: Normalized coefficient matrix from RG flow
        n_max: Maximum Fourier mode
        
    Returns:
        Estimated Jv value
    """
    # Extract key coefficients for estimation
    center_idx = n_max
    lambda_10 = final_matrix[center_idx+1, center_idx].real
    lambda_20 = final_matrix[center_idx+2, center_idx].real
    
    # Only proceed if both coefficients are non-zero
    if abs(lambda_10) > 1e-10 and abs(lambda_20) > 1e-10:
        # Calculate ratio and estimate Jv
        ratio = abs(lambda_20 / lambda_10)
        
        # For normal decay, λ₍₂,₀₎ should be smaller than λ₍₁,₀₎
        if ratio < 1:
            Jv = -3 / (2 * np.log(ratio))  # From the ratio of λ₍₂,₀₎/λ₍₁,₀₎ = exp(-3/2Jv)
        else:
            # For inverted cases, adjust the formula
            Jv = 3 / (2 * np.log(ratio))
            
        return Jv
    
    return None

def create_coefficient_comparison_table(J, M, n_max=20, n_steps=50, b=2, d=2, decimals=6):
    """
    Generate a table comparing RG flow coefficients with Villain fixed-line values
    
    Args:
        J: Two-spin coupling
        M: Four-spin coupling
        n_max: Maximum Fourier mode
        n_steps: Number of RG steps
        b: Length rescaling factor
        d: Dimension
        decimals: Number of decimal places to display
        
    Returns:
        Table as string
    """
    # Generate RG flow and get final coefficient matrix
    final_matrix = generate_rg_flow(J, M, n_max, n_steps, b, d)
    
    # Define unique coefficients to display
    # Use a list of tuples for consistent ordering
    center_idx = n_max
    coeff_indices = [
        (0, 0),   # (0,0)
        (1, 0),   # (1,0)
        (1, 1),   # (1,1)
        (2, 0),   # (2,0)
        (2, 1),   # (2,1)
        (2, 2),   # (2,2)
        (3, 0),   # (3,0)
        (3, 3),   # (3,3)
        (4, 0),   # (4,0)
        (4, 4)    # (4,4)
    ]
    
    # Extract coefficients from the matrix
    rg_coeffs = {}
    for n, m in coeff_indices:
        n_idx = center_idx + n
        m_idx = center_idx + m
        if 0 <= n_idx < final_matrix.shape[0] and 0 <= m_idx < final_matrix.shape[1]:
            rg_coeffs[(n, m)] = final_matrix[n_idx, m_idx].real
    
    # Estimate Jv from the coefficient matrix
    estimated_Jv = estimate_Jv_from_coefficients(final_matrix, n_max)
    
    # Calculate Villain fixed-line coefficients
    villain_coeffs = {}
    if estimated_Jv is not None:
        villain_coeffs = calculate_villain_coefficients(estimated_Jv, coeff_indices, n_max)
    
    # Create table headers
    headers = ["Source", "λ(0,0)", "λ(1,0)", "λ(1,1)", "λ(2,0)", "λ(2,1)", 
               "λ(2,2)", "λ(3,0)", "λ(3,3)", "λ(4,0)", "λ(4,4)"]
    
    # Create table rows
    rg_row = ["RG Flow"]
    villain_row = ["Villain (Jv={:.3f})".format(estimated_Jv if estimated_Jv is not None else 0)]
    
    # Fill in coefficient values
    for n, m in coeff_indices:
        # Add RG coefficient
        if (n, m) in rg_coeffs:
            rg_row.append("{:.{prec}f}".format(rg_coeffs[(n, m)], prec=decimals))
        else:
            rg_row.append("N/A")
        
        # Add Villain coefficient
        if (n, m) in villain_coeffs:
            villain_row.append("{:.{prec}f}".format(villain_coeffs[(n, m)], prec=decimals))
        else:
            villain_row.append("N/A")
    
    # Create table
    table_data = [rg_row, villain_row]
    table = tabulate(table_data, headers=headers, tablefmt="grid")
    
    # Print parameters
    param_info = f"Parameters: J={J}, M={M}, n_steps={n_steps}, b={b}, d={d}\n"
    full_table = param_info + table
    
    return full_table


# Create table

In [9]:
table = create_coefficient_comparison_table(J=1.5, M=0.5, n_max=20, n_steps=10, b=3, d=2, decimals=3)
print(table)

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

Parameters: J=1.5, M=0.5, n_steps=10, b=3, d=2
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Source             |   λ(0,0) |   λ(1,0) |   λ(1,1) |   λ(2,0) |   λ(2,1) |   λ(2,2) |   λ(3,0) |   λ(3,3) |   λ(4,0) |   λ(4,4) |
+====================+==========+==========+==========+==========+==========+==========+==========+==========+==========+==========+
| RG Flow            |        1 |    0.837 |      0.7 |    0.491 |    0.411 |    0.241 |    0.201 |    0.041 |    0.058 |    0.003 |
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
| Villain (Jv=2.808) |        1 |    0.837 |      0.7 |    0.491 |    0.411 |    0.241 |    0.201 |    0.041 |    0.058 |    0.003 |
+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
